In [5]:
import datetime
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import quandl
import pickle
import requests
api_key=open('datasets/key.txt','r').read()

In [9]:
def mortgage_30y():
    df = quandl.get("FMAC/MORTG", trim_start="1975-01-01", authtoken=api_key)
    df["Value"] = (df["Value"]-df["Value"][0]) / df["Value"][0] * 100.0
    df=df.resample('M').mean()
    print(df.head())
    return df

mortgage_30y()

               Value
Date                
1975-01-31  0.000000
1975-02-28 -3.393425
1975-03-31 -5.620361
1975-04-30 -6.468717
1975-05-31 -5.514316


,Value
Date,
1975-01-31,0.000000
1975-02-28,-3.393425
1975-03-31,-5.620361
1975-04-30,-6.468717
1975-05-31,-5.514316
1975-06-30,-5.726405
1975-07-31,-5.726405
1975-08-31,-5.196182
1975-09-30,-3.181336


In [10]:
HPI_data = pd.read_pickle('datasets/fiddy_states4.pickle')

In [14]:
def HPI_Benchmark():
    df_bmark = quandl.get("FMAC/HPI_USA", authtoken=api_key)
    df_bmark.rename(columns={'NSA Value':'US' + '_NSA Value' , 'SA Value' : 'US' + '_SA Value'},inplace=True)
    df_bmark['US' + '_NSA Value'] = (df_bmark['US' + '_NSA Value']-df_bmark['US' + '_NSA Value'][0]) /                 df_bmark['US' + '_NSA Value'][0] * 100.0
    df_bmark['US' + '_SA Value'] = (df_bmark['US' + '_SA Value']-df_bmark['US' + '_SA Value'][0]) / df_bmark['US'+ '_SA Value'][0] * 100.0
    
    return df_bmark

HPI_Bench=HPI_Benchmark()

In [12]:
m30 = mortgage_30y()

               Value
Date                
1975-01-31  0.000000
1975-02-28 -3.393425
1975-03-31 -5.620361
1975-04-30 -6.468717
1975-05-31 -5.514316


In [13]:
m30.columns=['M30']

In [15]:
HPI = HPI_Bench.join(m30)
print(HPI.head())

            US_NSA Value  US_SA Value       M30
Date                                           
1975-01-31      0.000000     0.000000  0.000000
1975-02-28      0.633958     0.654468 -3.393425
1975-03-31      1.686294     1.585822 -5.620361
1975-04-30      3.055918     2.724932 -6.468717
1975-05-31      3.876675     3.329405 -5.514316


In [16]:
print(HPI.corr())

              US_NSA Value  US_SA Value       M30
US_NSA Value      1.000000     0.999810 -0.777941
US_SA Value       0.999810     1.000000 -0.778356
M30              -0.777941    -0.778356  1.000000


In [17]:
state_HPI_M30 = HPI_data.join(m30)
print(state_HPI_M30.corr())

              AL NSA Value  AL SA Value  AK NSA Value  AK SA Value       M30
AL NSA Value      1.000000     0.999792      0.954640     0.954408 -0.795838
AL SA Value       0.999792     1.000000      0.954433     0.954591 -0.796195
AK NSA Value      0.954640     0.954433      1.000000     0.999781 -0.734754
AK SA Value       0.954408     0.954591      0.999781     1.000000 -0.735224
M30              -0.795838    -0.796195     -0.734754    -0.735224  1.000000


In [18]:
print(state_HPI_M30.corr()['M30'])

AL NSA Value   -0.795838
AL SA Value    -0.796195
AK NSA Value   -0.734754
AK SA Value    -0.735224
M30             1.000000
Name: M30, dtype: float64


In [19]:
print(state_HPI_M30.corr()['M30'].describe())

count    5.000000
mean    -0.412402
std      0.790146
min     -0.796195
25%     -0.795838
50%     -0.735224
75%     -0.734754
max      1.000000
Name: M30, dtype: float64


In [23]:
# def sp500_data():
#    df = quandl.get("YAHOO/INDEX_GSPC", trim_start="1975-01-01", authtoken=api_key)
#    df["Adjusted Close"] = (df["Adjusted Close"]-df["Adjusted Close"][0]) / df["Adjusted Close"][0] * 100.0
#   df=df.resample('M').mean()
#    df.rename(columns={'Adjusted Close':'sp500'}, inplace=True)
#   df = df['sp500']
#    return df
def sp500_data():
    df = quandl.get("YALE/SPCOMP", trim_start="1975-01-01", authtoken=api_key)
    df.columns = ['Adjusted Close',"1","1","1","1","1","1","1","1"]
    df["Adjusted Close"] = (df["Adjusted Close"]-df["Adjusted Close"][0]) / df["Adjusted Close"][0] * 100.0
    df=df.resample('M').mean()
    df.rename(columns={'Adjusted Close':'sp500'}, inplace=True)
    df = df['sp500']
    return df

# def gdp_data():
#    df = quandl.get("BCB/4385", trim_start="1975-01-01", authtoken=api_key)
#    df["Value"] = (df["Value"]-df["Value"][0]) / df["Value"][0] * 100.0
#    df=df.resample('M').mean()
#    df.rename(columns={'Value':'GDP'}, inplace=True)
#    df = df['GDP']
#    return df
def gdp_data():
    df = quandl.get("FRED/GDP", trim_start="1975-01-01", authtoken=api_key)
    df["Value"] = (df["Value"]-df["Value"][0]) / df["Value"][0] * 100.0
    df=df.resample('M').mean()
    df.rename(columns={'Value':'GDP'}, inplace=True)
    df = df['GDP']
    return df

#def us_unemployment():
#    df = quandl.get("ECPI/JOB_G", trim_start="1975-01-01", authtoken=api_key)
#    df["Unemployment Rate"] = (df["Unemployment Rate"]-df["Unemployment Rate"][0]) / df["Unemployment Rate"][0] * 100.0
#   df=df.resample('1D').mean()
#    df=df.resample('M').mean()
#   return df
def us_unemployment():
    df = quandl.get("FRED/UNRATE", trim_start="1975-01-01", authtoken=api_key)
    df.columns = ["Unemployment Rate"]
    df["Unemployment Rate"] = (df["Unemployment Rate"]-df["Unemployment Rate"][0]) / df["Unemployment Rate"][0] * 100.0
    df=df.resample('1D').mean()
    df=df.resample('M').mean()
    return df

In [24]:
sp500 = sp500_data()

In [25]:
gdp = gdp_data()

In [26]:
unemployment = us_unemployment()

In [27]:
HPI = HPI_Bench.join([m30,sp500,gdp,unemployment])

In [28]:
HPI.dropna(inplace=True)

In [29]:
print(HPI.corr())

                   US_NSA Value  US_SA Value       M30     sp500       GDP  \
US_NSA Value           1.000000     0.999802 -0.774121  0.906259  0.959958   
US_SA Value            0.999802     1.000000 -0.774606  0.906262  0.960100   
M30                   -0.774121    -0.774606  1.000000 -0.801612 -0.832021   
sp500                  0.906259     0.906262 -0.801612  1.000000  0.949044   
GDP                    0.959958     0.960100 -0.832021  0.949044  1.000000   
Unemployment Rate     -0.293698    -0.293726  0.246204 -0.366263 -0.196497   

                   Unemployment Rate  
US_NSA Value               -0.293698  
US_SA Value                -0.293726  
M30                         0.246204  
sp500                      -0.366263  
GDP                        -0.196497  
Unemployment Rate           1.000000  


In [30]:
 HPI.to_pickle('datasets/HPI.pickle')